In [11]:
user = "Alex"
pathtorepo = "C:\\Users\\" * user * "\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [12]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, Statistics

In [13]:
function HR_mem(u, p, t)
    function sigma(x)
        return 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
    end
    memristor(z, k1_me, k2_me) = k1_me + k2_me * z^2

    a, b, c, d, s, xr, r,  I, vs, k1, k2, k1_me, k2_me  = p
    x1, y1, z1, x2, y2, z2, z = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - k1 * ( x1 - vs ) * sigma(x2) + memristor(z, k1_me, k2_me)*(x2 - x1)
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - k2 * ( x2 - vs ) * sigma(x1) + memristor(z, k1_me, k2_me)*(x1 - x2)
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )

    du7 = x1 - x2
    
    return SVector(du1, du2, du3, du4, du5, du6, du7)
end

function save_tofile(name, Λs, u0s, eva)
    nameLS = "LSs "*  name * ".jld"
    nameu0s = "u0s "*  name * ".jld"
    nameeva = "EEs "*  name * ".jld"
    save(nameLS, "data", Λs)
    save(nameu0s, "data", u0s)
    save(nameeva, "data", EEs)
end

function output(idx, parameter, u0)
    println("index: $idx; k2_me: $parameter");flush(stdout)
    println("u0: $u0");flush(stdout)
end
function output(Λ, u0)
    println("Λ: $Λ");flush(stdout)
    println("last point: $u0");flush(stdout)
end
function separate()
    println("----------------");flush(stdout)
    println("  ");flush(stdout)
end

separate (generic function with 1 method)

In [14]:
a = 1.0; b = 3.0; c = 1.0; d = 5.0;
xr = -1.6; r = 0.01; s = 5.0; I = 4.0; xv = 2.0;
k1= 0.07; k2 = 0.07;
k1_me = 0.0
k2_me = 0.0
u0_start = [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0, 0.0]
p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me, k2_me];

In [15]:
tt = 1500
t = 1500
tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

(alg = RK4(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)), adaptive = false, dt = 0.001)

In [16]:
u0 = [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0, 0.0]
k1me_start = 0.0
k1me_end = 1.0
k1me_length = 300

k1me_range = range(k1me_start, k1me_end, length = k1me_length)

0.0:0.0033444816053511705:1.0

In [17]:
Λs =  zeros(length(k1me_range), 7)
u0s = zeros(length(k1me_range), 7)
EEs = zeros(length(k1me_range));

In [18]:
filename = "300 excitatory change k1_me to 1"

"300 excitatory change k1_me to 1"

In [19]:
for (idx, k1_me_) in enumerate(k1me_range)

    if idx == 1
        global u0 = u0_start
    end

    #output(idx, k2me_, u0)

    p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me_, k2_me]

    prob = ODEProblem(HR_mem, u0, (0, tt), p)
    sol = solve(prob, RK4(), adaptive = false, dt = 0.001)

    #ds = CoupledODEs(HR_mem, u0, p, diffeq = integ_set)
    #tr, _ = trajectory(ds, t; Δt = 0.01, Ttr = tt)

    u0 = sol[end]
    prob = ODEProblem(HR_mem, u0, (0, t), p)
    sol = solve(prob, RK4(), adaptive = false, dt = 0.001)

    ds = CoupledODEs(HR_mem,  u0, p, diffeq = integ_set)
    Λ = lyapunovspectrum(ds, t)
    Λ = sort(Λ)
    Λs[idx, :] = Λ
    u0s[idx, :] = u0

   

    if Λ[7] > 0
        xsum = sol[1, :] + sol[4, :]
        Hs = Statistics.mean(xsum) + 6 * Statistics.std(xsum)
        counts = xsum[xsum.>=Hs]
        counts = length(counts)
        EEs[idx]  = counts
        save_tofile(filename, Λs, u0s, counts)
    end
    # output(Λs[idx, :], u0s[idx, :])
    # separate()
    
    
end